## Analysis of Player Purchasing in "Heroes of Pymoli"
by:  Andrew Guenthner
Date:  20 Mar 2019

Key findings:

* Purchasing behavior follows player demongraphics, purchasers are largely male with a strong plurality of 20-24 year-olds

* Analysis of top purchasers and most popular items shows modest inequality -- the top 1% of purchasers spend at 3-4 x average

* The best revenue producers tended to be the most popular items with above-average prices, rather than discount items

Notes:  To run the analyses, the input file "purchase_data.csv" needs to be in a folder
called "Resources" in the same directory as this notebook.  

An accompanying "README" file should also be available in the same directory as this
notebook.  

In [1]:
# Import dependencies
import pandas as pd

# Set up file input
purchase_data_source = 'Resources/purchase_data.csv'
purchase_df = pd.read_csv(purchase_data_source)

In [2]:
# Check file header to make sure the DataFrame has loaded correctly
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


These data sets are expected to be clean and ready-to-go.  To verify, a simple check
for any missing values will suffice.

In [3]:
# Quick check for any missing values ...
purchase_df.isnull().sum()

Purchase ID    0
SN             0
Age            0
Gender         0
Item ID        0
Item Name      0
Price          0
dtype: int64

In [4]:
# Note the data types of each column for reference.
purchase_df.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

### Initial exploratory analysis

Some basic characteristics of the data: 

1) Total number of players in the dataset (assuming one screen name per player)

2) Total number of items purchased, the average purchase price, total number of purchases, and total revenue 

In [5]:
# Compute total number of unique screen names, total items, average price, number of transactions, and total revenue
total_players = purchase_df.SN.nunique()
total_items = purchase_df['Item Name'].nunique()
average_price = f'${purchase_df.Price.mean():.2f}'
total_purchases = purchase_df['Purchase ID'].count()
total_revenue = f'${purchase_df.Price.sum():,.2f}'
# Generate a DataFrame to provide this information in a nicely formatted table
summary_df = pd.DataFrame({'Total Players':total_players,'Total Items':total_items,'Average Price':average_price,
                           'Total Purchases':total_purchases,'Total Revenue':total_revenue},index = [0])
summary_df.style.hide_index()

Total Players,Total Items,Average Price,Total Purchases,Total Revenue
576,179,$3.05,780,"$2,379.77"


This is a reasonable size group for drawing inferences from the total set, but small enough that
we may need to be careful about analyzing subsets, such as particular age groups.

### Gender Analysis

The breakdown of the data by gender is as follows:

1) Players by gender (based on unique screen name count)

In [15]:
# Make a general-purpose group-by item
purch_by_gender = purchase_df.groupby('Gender')
# Count unique screen names by group
gender_count = pd.DataFrame(purch_by_gender['SN'].nunique().reset_index())
# Generate % data and format the dataframe 
gender_count['% of Players'] = 100 * gender_count['SN'] / gender_count['SN'].sum()
gender_count = gender_count.rename(columns={'SN':'Player Count'})
gender_count['% of Players'] = gender_count['% of Players'].map('{:.2f}'.format)
gender_count.style.hide_index()

Gender,Player Count,% of Players
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


The players identify as male by a large margin.

2) Purchase characteristics by gender, including number of purchases, average purchase amount, total purchases by each gender, and average spent on purchases per person by gender

In [7]:
# Count transactions by gender and start a new DataFrame
purch_count_gender = pd.DataFrame(purch_by_gender['Purchase ID'].count().reset_index())
# Average transaction amount by gender
price_avg_gender = pd.DataFrame(purch_by_gender['Price'].mean().reset_index())
# Total spend by gender
price_total_gender = pd.DataFrame(purch_by_gender['Price'].sum().reset_index())
# Merge these into a new dataframe for further analysis, and rename columns
merge_by_gender1 = purch_count_gender.merge(price_avg_gender,how='outer',on='Gender')
merge_by_gender2 = merge_by_gender1.merge(price_total_gender,how='outer',on='Gender')\
.rename(columns={'Purchase ID':'Transactions','Price_x':'Average Transaction','Price_y':'Total Spent'})
# Compute the average spent per player
merge_by_gender2['Average Spent per Player'] = merge_by_gender2['Total Spent'] / gender_count['SN']
# Format for display
merge_by_gender2['Average Transaction'] = merge_by_gender2['Average Transaction'].map('${:,.2f}'.format)
merge_by_gender2['Total Spent'] = merge_by_gender2['Total Spent'].map('${:,.2f}'.format)
merge_by_gender2['Average Spent per Player'] = merge_by_gender2['Average Spent per Player'].map('${:,.2f}'.format)
merge_by_gender2.style.hide_index()

Gender,Transactions,Average Transaction,Total Spent,Average Spent per Player
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


Players identifying as male spent slightly less than those identifying as other genders, on both a per transaction and total per player basis.  However, because of the larger numbers, they represent by far the largest source of revenue.

Game players overwhelmingly identify as male.  The following table compares purchase behavior by gender.

### Age-Group Analysis

For age-based analysis, the purchase data was binned into five-year intervals, starting with under 10, 10-14, 15-19 ... ending with 40 and above (the maximum player age was 45).

In [8]:
# Set up bind and labels for age groups
bins = [0, 10, 15, 19, 24, 29, 34, 39, 46]
age_labels = ['Under 10','10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40 and over']
# Create age groups designation and append to DataFrame
purchase_df['Age Group'] = pd.cut(purchase_df.Age,bins,labels=age_labels)
# Create groupby with age groups 
purch_by_agegroup = purchase_df.groupby('Age Group')
# Start a new DataFrame with a re-labeled column head, showing count of items purchased by age group
purchase_byage_df = pd.DataFrame(purch_by_agegroup['Purchase ID'].count().reset_index().rename(columns={"Purchase ID":"Items Purchased"}))
# Make new columns with average price and total spend by age group
merge3 = purchase_byage_df.merge(pd.DataFrame(purch_by_agegroup['Price'].mean().rename('Average Item Price')), on='Age Group', how='outer')
purchase_byage_df = merge3.merge(pd.DataFrame(purch_by_agegroup['Price'].sum().rename('Total Revenue')), on='Age Group', how='outer')
# Make one new column with average spend per person
perperson_spend_by_age = purchase_df.groupby('Age Group').apply(lambda grp : grp.Price.sum() / grp.SN.nunique())
purchase_byage_df = purchase_byage_df.merge(pd.DataFrame(perperson_spend_by_age.rename('Revenue per Person')), on='Age Group', how='outer')
# Clean up the column formats for display 
purchase_byage_df['Average Item Price'] = purchase_byage_df['Average Item Price'].map('${:,.2f}'.format)
purchase_byage_df['Total Revenue'] = purchase_byage_df['Total Revenue'].map('${:,.2f}'.format)
purchase_byage_df['Revenue per Person'] = purchase_byage_df['Revenue per Person'].map('${:,.2f}'.format)
purchase_byage_df.style.hide_index()

Age Group,Items Purchased,Average Item Price,Total Revenue,Revenue per Person
Under 10,32,$3.40,$108.96,$4.54
10-14,54,$2.90,$156.60,$3.82
15-19,101,$3.04,$307.24,$3.79
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40 and over,13,$2.94,$38.24,$3.19


Almost 50% of revenue was generated by 20-24 years olds, although on a per-person basis, their spending was not especially large.  On a per-person basis, players aged 35-39 outspent other age groups by about 20% per purchase, and players under 10 were also associated with a higher average item price and revenue per person.  

### Top purchasers, most popular items, and largest sources of revenue

The table below shows the key characteritics of the five players with the largest total purchase values.

In [9]:
# Generate the groupby DataFrame
top5_players = purchase_df.groupby('SN').agg({'Purchase ID':'count','Price':['mean','sum']}).reset_index()
# Sort and chop to top 5
top5_players = top5_players.sort_values([('Price','sum')],ascending=False).head(5)
# Prettify with new, single-level column names and formats
top5_players.columns = top5_players.columns.get_level_values(0)
top5_players.columns = ['SN','Items Purchased','Average Price','Total Spent']
top5_players['Average Price'] = top5_players['Average Price'].map('${:,.2f}'.format)
top5_players['Total Spent'] = top5_players['Total Spent'].map('${:,.2f}'.format)
# Display
top5_players.style.hide_index()

SN,Items Purchased,Average Price,Total Spent
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In this data set, the top 1% of spenders outspent the average player by a factor of roughly 3-4.   

The table below shows the key characteritics of the five most popular items.

In [10]:
# Generate the groupby DataFrame
top5_items = purchase_df.groupby('Item ID').agg({'Item Name':'first','Purchase ID':'count','Price':['mean','sum']}).reset_index()
# Sort and chop to top 5
top5_items = top5_items.sort_values([('Purchase ID','count')],ascending=False).head(5)
# Prettify with new, single-level column names and formats
top5_items.columns = top5_items.columns.get_level_values(0)
top5_items.columns = ['Item ID','Item Name','Times Purchased', 'Average Price','Total Spent']
top5_items['Average Price'] = top5_items['Average Price'].map('${:,.2f}'.format)
top5_items['Total Spent'] = top5_items['Total Spent'].map('${:,.2f}'.format)
# Display
top5_items.style.hide_index()

Item ID,Item Name,Times Purchased,Average Price,Total Spent
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


 In this data set, the top five items are purchased 2-3 x as often as the "average" (780 transactions for 179 items yields an average of somewhat over four purchases per item)

The table below shows the key characteritics of the five items that generated the most revenue.

In [11]:
# Generate the groupby DataFrame
top5_sales = purchase_df.groupby('Item ID').agg({'Item Name':'first','Purchase ID':'count','Price':['mean','sum']}).reset_index()
# Sort and chop to top 5
top5_sales = top5_sales.sort_values([('Price','sum')],ascending=False).head(5)
# Prettify with new, single-level column names and formats
top5_sales.columns = top5_sales.columns.get_level_values(0)
top5_sales.columns = ['Item ID','Item Name','Times Purchased', 'Average Price','Total Spent']
top5_sales['Average Price'] = top5_sales['Average Price'].map('${:,.2f}'.format)
top5_sales['Total Spent'] = top5_sales['Total Spent'].map('${:,.2f}'.format)
# Display
top5_sales.style.hide_index()

Item ID,Item Name,Times Purchased,Average Price,Total Spent
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


In this data set, the top five items by revenue generated 3-4 x the average revenue per item (\\$2,379.77 total revenue over 179 unique items yields an average of about \\$13.30 per item.  These items all have a higher than average price, so less
expensive items with higher demand do not appear to produce high revenues.  On the other hand, three of the five most popular
items are also in the top five revenue producers.  It would appear that making higher-priced items more popular has resulted in maximizing revenue per item.  